In [10]:
import index_flow as idf

In [11]:
idf.about()

This module contains functions to import and QC indexed fcs files. For more info type index_flow.about?


In [12]:
idf.get_comp_data??

In [46]:
def get_channels(directory):
    
    '''
    This functions prints the sort channels used in each index plate and returns a dictionary of the channels used for each plate
    
    '''
    out_dict = {}
    for count, filename in enumerate(os.listdir(directory)):
        fn = os.path.join(directory, filename)
        plateid = (fn.split('/'))[4]

        meta, data = fcsparser.parse(fn, reformat_meta=True)

        comp_fields = meta['SPILL'].split(',')
        n = int(comp_fields[0])
        channels = comp_fields[1: n+1]

        out_dict[plateid] = channels
        
    print(out_dict)    
    
    return out_dict



In [47]:
get_channels(directory)

{'Sample_INX_Enriched2_001_016.fcs': ['YG582/15-A', 'YG670/30-A', 'YG780/60-A', 'V450/50-A', 'V610/20-A', 'B530/30-A', 'R660/20-A', 'R780/60-A'], 'Sample_INX_Enriched2_002_017.fcs': ['YG582/15-A', 'YG670/30-A', 'YG780/60-A', 'V450/50-A', 'V610/20-A', 'B530/30-A', 'R660/20-A', 'R780/60-A'], 'Sample_INX_Enriched2_003_018.fcs': ['YG582/15-A', 'YG670/30-A', 'YG780/60-A', 'V450/50-A', 'V610/20-A', 'B530/30-A', 'R660/20-A', 'R780/60-A'], 'Sample_INX_Enritched_001_014.fcs': ['YG582/15-A', 'YG670/30-A', 'YG780/60-A', 'V450/50-A', 'V610/20-A', 'B530/30-A', 'R660/20-A', 'R780/60-A'], 'Sample_INX_Enritched_002_015.fcs': ['YG582/15-A', 'YG670/30-A', 'YG780/60-A', 'V450/50-A', 'V610/20-A', 'B530/30-A', 'R660/20-A', 'R780/60-A']}


('Sample_INX_Enritched_002_015.fcs',
 ['YG582/15-A',
  'YG670/30-A',
  'YG780/60-A',
  'V450/50-A',
  'V610/20-A',
  'B530/30-A',
  'R660/20-A',
  'R780/60-A'])

In [9]:
import os
import sys
import math
import numpy as np
import numpy.linalg
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import fcsparser

In [14]:
directory = '../Data/GR_BM/Indexed/'
#Make plate key - note one plate split over 2 index files
files = []
for filename in os.listdir(directory):
    files.append(filename)

plates = ['AS-223', 'AS-221','enriched2_3', 'AS-218', 'enriched'] 

plate_key = dict(zip(files, plates))

channel_key = {'YG582/15-A': 'CD34-PE', 
               'YG670/30-A': 'Lin-PE-Cy5', 
               'YG780/60-A': 'CD123-PE-Cy7', 
               'V450/50-A': 'CD90-BV421', 
               'V610/20-A': 'Zombie', 
               'B530/30-A': 'CD45RA-FITC', 
               'R660/20-A': 'IL1RAP-APC', 
               'R780/60-A': 'CD38-APC-cy7'
}

In [16]:
def get_inx(meta : dict):  
    
    '''
    This function retrieves the wells with sort data from an already imported indexed fcs file.
    get_inx(meta : dict)
    meta is an output from meta, data =  fcsparser.parse(filename, reformat_meta=True)
    The function returns the sort locations as a list, using the default software assignment x(0-23), y(0-15) for Aria
    '''
    
    i = 1
    key = f'INDEX SORTING LOCATIONS_{i}'
    sort_locs = [] #Sort locations from the metadata, numerical row then column starting at 0
    while key in meta:
        sort_locs.append(meta[key])
        i += 1
        key = f'INDEX SORTING LOCATIONS_{i}'
    sort_locs = ''.join(sort_locs).split(';')
    return sort_locs

def count_files(directory): 
    
    '''
    This function counts the number of files in an input directory.
    count_files(directory)
    The function returns the number of files.
    '''
    
    count = 0
    for filename in os.listdir(directory):
        count += 1
        
    return count    


In [17]:
get_comp_data(directory, plate_key, channel_key)

['YG582/15-A', 'YG670/30-A', 'YG780/60-A', 'V450/50-A', 'V610/20-A', 'B530/30-A', 'R660/20-A', 'R780/60-A'] [7, 8, 9, 10, 11, 12, 13, 14]


,FSC-A,FSC-W,FSC-H,SSC-A,SSC-W,SSC-H,CD34-PE,Lin-PE-Cy5,CD123-PE-Cy7,CD90-BV421,Zombie,CD45RA-FITC,IL1RAP-APC,CD38-APC-cy7,Time,Well,Plate,Plate_Well
Sorted well,,,,,,,,,,,,,,,,,,
A2,200734.187500,113981.734375,115416.0,70003.679688,88088.960938,52081.0,2723.716158,44.567666,222.955254,2692.214222,551.518263,300.248769,278.779625,3385.081015,1139.800049,A2,AS-223,AS-223_A2
A3,139177.796875,100187.359375,91041.0,45485.339844,84249.820312,35382.0,955.617626,222.908222,104.368045,229.181269,250.838521,58.123457,-313.489328,5015.439829,1391.099976,A3,AS-223,AS-223_A3
A4,130119.296875,93257.851562,91440.0,38682.808594,80723.343750,31405.0,1563.457633,156.605632,32.961283,-96.358526,115.990621,-6.153647,-328.752601,15077.365164,1696.500000,A4,AS-223,AS-223_A4
A5,141398.093750,103347.640625,89665.0,42408.148438,85734.656250,32417.0,1191.773374,823.896315,161.039370,91.875548,341.682826,673.338688,79.936395,8615.739655,1986.000000,A5,AS-223,AS-223_A5
A6,101859.296875,88245.921875,75646.0,32296.140625,74750.492188,28315.0,1051.540190,142.112702,83.699411,82.248799,159.927657,73.287048,82.276964,2830.498176,2216.300049,A6,AS-223,AS-223_A6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P21,133685.093750,98549.921875,88901.0,63264.660156,84412.992188,49117.0,1006.344820,23.430623,303.816242,58.293960,271.860689,281.153117,49.102138,10516.791125,80972.500000,P21,enriched,enriched_P21
P20,174359.703125,106367.398438,107428.0,64519.199219,88895.828125,47565.0,2776.700837,224.725449,9.118733,166.498091,254.623467,180.873802,428.097517,11393.715174,81206.601562,P20,enriched,enriched_P20
P19,174377.703125,114513.781250,99796.0,93659.851562,91879.351562,66806.0,1828.468616,146.443938,85.997391,-4.482140,526.930382,370.151417,121.674131,12163.353478,81395.000000,P19,enriched,enriched_P19


In [7]:
import pandas as pd

In [9]:
pd.DataFrame??

In [13]:
sourcefile = '../Data/Amp_data/allele_counts.tsv'
df = pd.read_csv(sourcefile, header = None, index_col = None, sep='\t')
df

,0,1,2,3,4,5,6,7,8,9,...,211,212,213,214,215,216,217,218,219,220
0,NaN,AS-209,AS-209,AS-209,AS-209,AS-209,AS-209,AS-209,AS-209,AS-196,...,AS-207,AS-207,AS-207,AS-207,AS-207,AS-207,AS-207,AS-207,AS-207,AS-207
1,NaN,EL_SRSF2,EL_SRSF2,EL_TET2a,EL_TET2a,EL_TET2b,EL_TET2b,EL_TGFB3_g,EL_TGFB3_g,RN_RUNX1_c,...,EL_SRSF2,EL_SRSF2,EL_TET2a,EL_TET2a,EL_TET2b,EL_TET2b,EL_TGFB3_c,EL_TGFB3_c,EL_TGFB3_g,EL_TGFB3_g
2,NaN,MT,WT,MT,WT,MT,WT,MT,WT,MT,...,MT,WT,MT,WT,MT,WT,MT,WT,MT,WT
3,A1,0,0,0,0,0,0,0,0,0,...,0,1,0,7,0,1,0,0,0,0
4,A2,0,0,2,1,0,0,0,0,0,...,1,2,4,4,0,3,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
382,P20,1,3,0,0,0,0,0,4,20,...,39,22,45,91,57,58,0,1,0,2
383,P21,0,4,0,1,0,0,0,2,0,...,10,16,47,59,32,8,0,0,0,2
384,P22,0,1,13,50,0,0,0,330,0,...,17,31,59,246,27,18,0,0,0,2
385,P23,1,248,0,24,0,1,0,6,1,...,5,16,802,240,3,1,0,0,1,1


In [19]:
df2 = df.T
amps = df2[1].drop_duplicates().to_list()
amps

[nan,
 'EL_SRSF2',
 'EL_TET2a',
 'EL_TET2b',
 'EL_TGFB3_g',
 'RN_RUNX1_c',
 'RN_RUNX1_g',
 'RN_SRSF2',
 'RN_TET2a',
 'RN_TET2b_c',
 'RN_TET2b_g',
 'EL_TGFB3_c',
 'GR_TET2a',
 'GR_TET2b',
 'EL_CUX1']

In [22]:
anon_key = {'EL':'PD7153','GR':'PD7151','RN':'JP001'}


In [39]:
new_amps = []
for a in amps[1:]:
    a = str(a)
    b = a.split('_', 1)
    c = anon_key[b[0]]
    d = c + '_' + b[1]
    new_amps.append(d)
    
new_amps    

['PD7153_SRSF2',
 'PD7153_TET2a',
 'PD7153_TET2b',
 'PD7153_TGFB3_g',
 'JP001_RUNX1_c',
 'JP001_RUNX1_g',
 'JP001_SRSF2',
 'JP001_TET2a',
 'JP001_TET2b_c',
 'JP001_TET2b_g',
 'PD7153_TGFB3_c',
 'PD7151_TET2a',
 'PD7151_TET2b',
 'PD7153_CUX1']

In [40]:
replace = dict(zip(amps[1:], new_amps))
replace

{'EL_SRSF2': 'PD7153_SRSF2',
 'EL_TET2a': 'PD7153_TET2a',
 'EL_TET2b': 'PD7153_TET2b',
 'EL_TGFB3_g': 'PD7153_TGFB3_g',
 'RN_RUNX1_c': 'JP001_RUNX1_c',
 'RN_RUNX1_g': 'JP001_RUNX1_g',
 'RN_SRSF2': 'JP001_SRSF2',
 'RN_TET2a': 'JP001_TET2a',
 'RN_TET2b_c': 'JP001_TET2b_c',
 'RN_TET2b_g': 'JP001_TET2b_g',
 'EL_TGFB3_c': 'PD7153_TGFB3_c',
 'GR_TET2a': 'PD7151_TET2a',
 'GR_TET2b': 'PD7151_TET2b',
 'EL_CUX1': 'PD7153_CUX1'}

In [41]:
df2

,0,1,2,3,4,5,6,7,8,9,...,377,378,379,380,381,382,383,384,385,386
0,NaN,NaN,NaN,A1,A2,A3,A4,A5,A6,A7,...,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24
1,AS-209,EL_SRSF2,MT,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,AS-209,EL_SRSF2,WT,0,0,3,0,0,0,0,...,0,1,1,3,0,3,4,1,248,4
3,AS-209,EL_TET2a,MT,0,2,0,0,0,0,0,...,0,1,0,0,0,0,0,13,0,0
4,AS-209,EL_TET2a,WT,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,50,24,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,AS-207,EL_TET2b,WT,1,3,9,4,5,1,1,...,2,27,8,5,20,58,8,18,1,5
217,AS-207,EL_TGFB3_c,MT,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
218,AS-207,EL_TGFB3_c,WT,0,0,0,0,0,0,0,...,0,3,0,0,0,1,0,0,0,0
219,AS-207,EL_TGFB3_g,MT,0,0,0,0,0,0,0,...,0,10,0,0,0,0,0,0,1,0


In [47]:
df3 = df2.copy()
df3[1] = df3[1].map(replace).fillna(df3[1])
df3

,0,1,2,3,4,5,6,7,8,9,...,377,378,379,380,381,382,383,384,385,386
0,NaN,NaN,NaN,A1,A2,A3,A4,A5,A6,A7,...,P15,P16,P17,P18,P19,P20,P21,P22,P23,P24
1,AS-209,PD7153_SRSF2,MT,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,AS-209,PD7153_SRSF2,WT,0,0,3,0,0,0,0,...,0,1,1,3,0,3,4,1,248,4
3,AS-209,PD7153_TET2a,MT,0,2,0,0,0,0,0,...,0,1,0,0,0,0,0,13,0,0
4,AS-209,PD7153_TET2a,WT,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,50,24,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,AS-207,PD7153_TET2b,WT,1,3,9,4,5,1,1,...,2,27,8,5,20,58,8,18,1,5
217,AS-207,PD7153_TGFB3_c,MT,0,0,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0
218,AS-207,PD7153_TGFB3_c,WT,0,0,0,0,0,0,0,...,0,3,0,0,0,1,0,0,0,0
219,AS-207,PD7153_TGFB3_g,MT,0,0,0,0,0,0,0,...,0,10,0,0,0,0,0,0,1,0


In [3]:
import index_haps

In [8]:
import pandas as pd

In [9]:
pd?